In [72]:
import requests
from bs4 import BeautifulSoup
import time
import random
import datetime
import numpy as np
import pandas as pd

In [73]:
def filesave(dataframe,name):
    dataframe.to_csv(name,sep="\t", encoding="utf-8",index=False)
def fileread(name):
    return pd.read_csv(name,sep="\t",encoding="utf-8")

In [74]:
def sbs_make_link_list(section_number,date,pageidx):
    url = "https://news.sbs.co.kr/news/newsSection.do?sectionType=0{}&pageDate={}&pageIdx={}".format(section_number,date,pageidx)
    raw = requests.get(url,
                   headers={'User-Agent':'Mozilla/5.0'})
    html = BeautifulSoup(raw.text, "html.parser")
    news_list=html.findAll("a",attrs={'class': 'news'})
    news_link_list = []
    if news_list:
        for i in news_list:
            news_link_list.append(str(i.attrs['href']))
    return news_link_list

In [75]:
def sbs_select_section_auto(sectionlist,startdate,stopdate):
    startdatetime = datetime.datetime(int(startdate[:4]),int(startdate[4:6]),int(startdate[6:]))
    stopdatetime = datetime.datetime(int(stopdate[:4]),int(stopdate[4:6]),int(stopdate[6:]))
    oneday=datetime.timedelta(days=1)
    autolinklist=[]
    for number in sectionlist:
        currentdatetime = startdatetime
        
        while currentdatetime < stopdatetime:
            d = currentdatetime.strftime("%Y%m%d")
            for idx in range(1,5):
                temp=sbs_make_link_list(number,d,idx)
                if not temp:
                    break
                autolinklist+=temp
            time.sleep(abs((np.random.randn())*0.1)+0.1)
            currentdatetime += oneday
            
    return autolinklist

In [165]:
def sbs_link_to_df(link_list):
    root_url="https://news.sbs.co.kr"
    data=[]
    for link in link_list:
        url = root_url+link
        res = requests.get(url,
                   headers={'User-Agent':'Mozilla/5.0'})
        
        pagehtml = BeautifulSoup(res.text, "html.parser")
        
        pagetitle=pagehtml.find("h3",attrs={'id': 'vmNewsTitle'})
        pagecontent=pagehtml.find("div",attrs={'class': 'text_area'})
        pagedate=pagehtml.find("span",attrs={'class': 'date'}).span
        pagesection = pagehtml.find("li",attrs={'class':'cate03'})
        
        if pagetitle and pagecontent and pagedate and pagesection:
            pagetitle=pagetitle.text
            pagecontent=pagecontent.text
            pagedate=pagedate.text
            pagesection=pagesection.text.replace("\n","")
            
        data.append([pagetitle,pagecontent,pagedate,pagesection,url])
        
        time.sleep(abs((np.random.randn())*0.05))
        
    return pd.DataFrame(data, columns=['title','content','date','section','url'])

In [84]:
#startdate를 읽어온 csv 파일에서 가장 최신 날짜로 자동으로 잡도록...하고 싶었지만 생각해보니 애초에 날짜 저장을 안 했다
def auto_save(startdate,day=3):
    url_df = fileread("sbs_news_url.csv")
    
    startdatetime = datetime.datetime(int(startdate[:4]),int(startdate[4:6]),int(startdate[6:]))
    oneday=datetime.timedelta(days=1)
    
    savedatetime = startdatetime + oneday*day
    savedate = savedatetime.strftime("%Y%m%d")
    
    s=sbs_select_section_auto([1,2,3,4,5,6,7,8,9],startdate,savedate)
    url_df_temp = pd.DataFrame(s,columns=['news_url'])
    url_df = pd.concat([url_df,url_df_temp])
    filesave(url_df,"sbs_news_url.csv")
    print(savedate,"까지 저장")

In [132]:
startdate = "20221012"
stopdate = "20221022"
startdatetime = datetime.datetime(int(startdate[:4]),int(startdate[4:6]),int(startdate[6:]))
oneday=datetime.timedelta(days=1)

In [138]:
# 30일간
for _ in range(10):
    auto_save(startdatetime.strftime("%Y%m%d"))
    
    startdatetime+=oneday*3

20221015 까지 저장
20221018 까지 저장
20221021 까지 저장


In [77]:
# s=sbs_select_section_auto([1,2,3,4,5,6,7,8,9],"20210920","20221022")
# url_df = pd.DataFrame(s,columns=['news_url'])
# filesave(url_df,"sbs_news_url.csv")

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [139]:
test=fileread("sbs_news_url.csv")
test

,news_url
0,/news/endPage.do?news_id=N1006903791
1,/news/endPage.do?news_id=N1006903790
2,/news/endPage.do?news_id=N1006903789
3,/news/endPage.do?news_id=N1006903687
4,/news/endPage.do?news_id=N1006903682
...,...
65485,/news/endPage.do?news_id=N1006939106
65486,/news/endPage.do?news_id=N1006939105
65487,/news/endPage.do?news_id=N1006939104
65488,/news/endPage.do?news_id=N1006939103


In [140]:
#중복으로 저장한 건 후처리로 걸러주기
print(len(test))
print(test['news_url'].unique())
test=pd.DataFrame(test['news_url'].unique(),columns=['news_url'])
print(len(test))
test

65490
['/news/endPage.do?news_id=N1006903791'
 '/news/endPage.do?news_id=N1006903790'
 '/news/endPage.do?news_id=N1006903789' ...
 '/news/endPage.do?news_id=N1006939104'
 '/news/endPage.do?news_id=N1006939103'
 '/news/endPage.do?news_id=N1006939123']
62475


,news_url
0,/news/endPage.do?news_id=N1006903791
1,/news/endPage.do?news_id=N1006903790
2,/news/endPage.do?news_id=N1006903789
3,/news/endPage.do?news_id=N1006903687
4,/news/endPage.do?news_id=N1006903682
...,...
62470,/news/endPage.do?news_id=N1006939106
62471,/news/endPage.do?news_id=N1006939105
62472,/news/endPage.do?news_id=N1006939104
62473,/news/endPage.do?news_id=N1006939103


In [141]:
filesave(test,"sbs_news_url.csv")

In [142]:
splitedlist=[test[i*100:i*100+100] for i in range(0,(len(test)//100)+1)]

In [144]:
splitedlist[0][:3]

,news_url
0,/news/endPage.do?news_id=N1006903791
1,/news/endPage.do?news_id=N1006903790
2,/news/endPage.do?news_id=N1006903789


In [153]:
splitedlist[1]

,news_url
100,/news/endPage.do?news_id=N1006909334
101,/news/endPage.do?news_id=N1006909333
102,/news/endPage.do?news_id=N1006909332
103,/news/endPage.do?news_id=N1006909331
104,/news/endPage.do?news_id=N1006909210
...,...
195,/news/endPage.do?news_id=N1006915390
196,/news/endPage.do?news_id=N1006915157
197,/news/endPage.do?news_id=N1006915272
198,/news/endPage.do?news_id=N1006915139


In [159]:
sbs_df = pd.DataFrame(columns=['title','content','date','section','url'])

In [167]:
count=60
for linklist in splitedlist[count:]:
    start = time.time()
    
    sbs_df_temp = sbs_link_to_df(list(linklist['news_url']))
    sbs_df = pd.concat([sbs_df,sbs_df_temp])
    filesave(sbs_df,"sbs_dataframe.csv")
    count+=1
    
    print("소요시간 :", time.time() - start)
    print(count*100,"번째까지 저장")
    
    time.sleep((random.random()*0.5)+0.5)

소요시간 : 37.40169095993042
6100 번째까지 저장
소요시간 : 39.26073122024536
6200 번째까지 저장
소요시간 : 38.124433279037476
6300 번째까지 저장
소요시간 : 37.4473032951355
6400 번째까지 저장
소요시간 : 40.43422985076904
6500 번째까지 저장
소요시간 : 38.81100821495056
6600 번째까지 저장
소요시간 : 39.33099317550659
6700 번째까지 저장
소요시간 : 37.12605047225952
6800 번째까지 저장
소요시간 : 39.03325891494751
6900 번째까지 저장
소요시간 : 36.9882447719574
7000 번째까지 저장
소요시간 : 40.17316269874573
7100 번째까지 저장
소요시간 : 37.50881886482239
7200 번째까지 저장
소요시간 : 41.47336149215698
7300 번째까지 저장
소요시간 : 37.23298525810242
7400 번째까지 저장
소요시간 : 38.617130517959595
7500 번째까지 저장
소요시간 : 37.42286729812622
7600 번째까지 저장
소요시간 : 39.82119393348694
7700 번째까지 저장
소요시간 : 38.482213258743286
7800 번째까지 저장
소요시간 : 41.44138169288635
7900 번째까지 저장
소요시간 : 37.36790060997009
8000 번째까지 저장
소요시간 : 38.11643719673157
8100 번째까지 저장
소요시간 : 37.23914885520935
8200 번째까지 저장
소요시간 : 38.175405979156494
8300 번째까지 저장
소요시간 : 37.834309577941895
8400 번째까지 저장
소요시간 : 37.35372567176819
8500 번째까지 저장
소요시간 : 37.08205485343933
8600 번째까지 저장
소요시간 : 38

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [ ]:
count=87
for _ in range(5):
    try:
        for linklist in splitedlist[count:]:
            start = time.time()

            sbs_df_temp = sbs_link_to_df(list(linklist['news_url']))
            sbs_df = pd.concat([sbs_df,sbs_df_temp])
            filesave(sbs_df,"sbs_dataframe.csv")
            count+=1

            print("소요시간 :", time.time() - start)
            print(count*100,"번째까지 저장")

            time.sleep((random.random()*0.5)+0.5)
    except:
        print(count,"에서 에러 발생")
        time.sleep(3)

소요시간 : 38.375858306884766
8800 번째까지 저장
소요시간 : 38.13387656211853
8900 번째까지 저장
소요시간 : 37.7824432849884
9000 번째까지 저장
소요시간 : 40.37823414802551
9100 번째까지 저장
소요시간 : 37.3439154624939
9200 번째까지 저장
소요시간 : 37.63626265525818
9300 번째까지 저장
소요시간 : 38.04548144340515
9400 번째까지 저장
소요시간 : 39.0338020324707
9500 번째까지 저장
소요시간 : 38.53817558288574
9600 번째까지 저장
소요시간 : 38.08000302314758
9700 번째까지 저장
소요시간 : 40.34846758842468
9800 번째까지 저장
소요시간 : 38.61404466629028
9900 번째까지 저장
소요시간 : 37.75872731208801
10000 번째까지 저장
소요시간 : 39.50932455062866
10100 번째까지 저장
소요시간 : 37.09375834465027
10200 번째까지 저장
소요시간 : 38.78402614593506
10300 번째까지 저장
소요시간 : 37.55732870101929
10400 번째까지 저장
소요시간 : 40.44399952888489
10500 번째까지 저장
소요시간 : 38.29832625389099
10600 번째까지 저장
소요시간 : 37.78064513206482
10700 번째까지 저장
소요시간 : 37.062418699264526
10800 번째까지 저장
소요시간 : 38.562161922454834
10900 번째까지 저장
소요시간 : 37.74899339675903
11000 번째까지 저장
소요시간 : 38.18002486228943
11100 번째까지 저장
소요시간 : 39.251559019088745
11200 번째까지 저장
소요시간 : 38.82300114631653
11300 번째까지 

In [161]:
sbs_df=fileread("sbs_dataframe.csv")

In [162]:
sbs_df

,title,content,date,section,url
0,러시아 점령지 첫 영토합병 주민투표 강행…이달 23~27일 예정,\n\n▲위 사진은 기사 내용과 관련이 없습니다.\n\n 친러시아 세력이 독립을 선...,2022.09.20 22:17,정치,https://news.sbs.co.kr/news/endPage.do?news_id...
1,중국 중앙은행 '사실상의 기준금리' LPR 동결,\n 중국이 사실상의 기준금리 역할을 하는 대출우대금리(LPR)를 동결했다고 중국...,2022.09.20 22:15,정치,https://news.sbs.co.kr/news/endPage.do?news_id...
2,"스웨덴, 미 연준 앞서 기준금리 1.0%p 파격 인상",\n 스웨덴 중앙은행이 자국 기준금리를 1.0% 포인트 인상했습니다.\n \n ...,2022.09.20 22:14,정치,https://news.sbs.co.kr/news/endPage.do?news_id...
3,"""정부 부실 대응"" 혼나고 내놓은 대책도 '부처 엇박자'",\n <앵커>\n \n 오늘(20일) 국회에서도 이번 스토킹 살인 사건은 막을 ...,2022.09.20 20:34,정치,https://news.sbs.co.kr/news/endPage.do?news_id...
4,"""너무 답답하십니다""…'조문 논란' 속 총리는 엉뚱 답변",\n <앵커>\n \n 오늘(20일) 국회 대정부 질문에서는 윤석열 대통령이 영...,2022.09.20 20:12,정치,https://news.sbs.co.kr/news/endPage.do?news_id...
...,...,...,...,...,...
3495,전자발찌 찬 범죄자가 심리상담?…상담자 성추행해 실형,\n 성폭행 혐의로 실형을 선고받고 전자발찌를 부착 중인 심리상담사가 이번에는 내...,2021.09.22 09:21,사회,https://news.sbs.co.kr/news/endPage.do?news_id...
3496,연휴 마지막 날 귀경길 정체…오후 4~5시 절정 예상,\n 추석 연휴 마지막 날인 오늘(22일) 아침부터 고속도로에 차량 행렬이 몰리면...,2021.09.22 08:50,사회,https://news.sbs.co.kr/news/endPage.do?news_id...
3497,"'중국판 리먼 사태' 일어날까…헝다, 파산 첫 고비",\n <앵커>\n \n 중국에서 가장 큰 민영 부동산 개발업체인 헝다의 파산 위...,2021.09.22 07:32,사회,https://news.sbs.co.kr/news/endPage.do?news_id...
3498,"휴게소서 화물차 연료선 '싹둑'…""파업 관련성 수사""",\n <앵커>\n \n 고속도로 휴게소에 주차된 화물차의 연료 공급선을 누군가 ...,2021.09.22 07:23,사회,https://news.sbs.co.kr/news/endPage.do?news_id...


In [ ]:
# 동아일보 쪽 코드 시험 안 해봄

In [ ]:
def donga_make_link_list(section_name,pageidx):
    url = "https://www.donga.com/news/{}?p={}&prod=news&ymd=&m=".format(section_name,pageidx)
    raw = requests.get(url,
                   headers={'User-Agent':'Mozilla/5.0'})
    html = BeautifulSoup(raw.text, "html.parser")
    news_list=html.findAll("span",attrs={'class': 'tit'})
    news_link_list=[]
    for i in news_list:
        if i.a:
            if "{}/article".format(section_name) in str(i.a.attrs['href']):
                news_link_list.append(str(i.a.attrs['href']))
    return news_link_list

In [ ]:
def donga_select_section_auto(sectionlist,stoppage):
    autolinklist=[]
    for section in sectionlist:
        for i in range(0,stoppage):
            idx=i*10+1
            temp=donga_make_link_list(section,idx)
            if not temp:
                break
            autolinklist+=temp
        time.sleep(abs((np.random.randn())*0.1)+0.1)

    return autolinklist

In [ ]:
def donga_link_to_df(link_list):
    data=[]
    for link in link_list:
        print(link, "로 페이지 요청...")
        res = requests.get(link,
                   headers={'User-Agent':'Mozilla/5.0'})
        
        pagehtml = BeautifulSoup(res.text, "html.parser")
        
        pagetitle=pagehtml.title
        pagecontent=pagehtml.find("div",attrs={'class': 'article_txt'})
        pagedate=pagehtml.find("span",attrs={'class': 'date01'})
        
        if pagetitle and pagecontent and pagedate:
            pagetitle=pagetitle.text
            r=pagecontent
            for i in pagecontent.find_all("span"):
                r=str(r).replace(str(i),"")
            rhtml = BeautifulSoup(r, "html.parser")
            split=rhtml.text.split("\n\n\n\n\n\n\n")
            pagecontent=split[0]
            pagedate=pagedate.text.replace("입력","")
            temp = url.split("/news/")
            temp = temp.split("?p")
            pagesection=temp[0]
            
        data.append([pagetitle,pagecontent,pagedate,pagesection,link])
        
        time.sleep(abs((np.random.randn())*0.1)+0.1)
        
    return pd.DataFrame(data, columns=['title','content','date','section','url'])

In [ ]:
donga_sections= ["Opinion","Politics","Economy","Inter","Society","Culture","Entertainment","Sports"]
d=donga_select_section_auto(donga_sections,1000)
url_df = pd.DataFrame(s,columns=['news_url'])
filesave(url_df,"donga_news_url.csv")